In [1]:
# Import libraries

import os
import time
import warnings
import numpy as np
from scipy import stats
import matplotlib as mpl
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

# Disble ignore warnings
warnings.filterwarnings('ignore')

In [2]:
# Global variables

# Number of samples
num_bins =10
num_samples_per_bin = 10
num_samples = 2*num_bins*num_samples_per_bin

# Number of shots
num_shots = 1024

# SA parameters
num_runs = 20
num_sa_iters = 50
ini_temp = 10
cool_coef = 0.95

# Create directory
today = date.today()
today = str(today)
today = "%s-%s-%s"%(today.split('-')[0],today.split('-')[1],today.split('-')[2])

date_dir = ['../data3/2021-10-29', '../data3/2021-10-30', '../data3/2021-10-31', '../data3/2021-11-01', '../data3/2021-11-02', '../data3/2021-11-03', '../data3/2021-11-04', '../data3/2021-11-05', '../data3/2021-11-06', '../data3/2021-11-07', '../data3/2021-11-08']
# date_dir = ['../data3/2021-11-08']
# if not os.path.exists(date_dir):
#     os.makedirs(date_dir)

## Original Normal DISQ_E2D classifier

In [4]:
# Discriminant number of parameters and bounds
# For Ell2D: origin-x, origin-y, radius-x, radius-y, angle of rotation X 2
num_params = 10
bounds = [[-5, 0], [10, 20], [0, 2*np.pi], [4, 8], [4, 8], [0, 10], [10, 20], [0, 2*np.pi], [2, 5], [2, 5]]

# Sample the performance of a configuration
def sample_perf(config, obj, train_d_1, train_o_p):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    
    A0 = (np.cos(config[2])/config[3])**2 + (np.sin(config[2])/config[4])**2
    B0 = 2 * np.cos(config[2]) * np.sin(config[2]) * ((1/config[3]**2) - (1/config[4]**2))
    C0 = (np.sin(config[2])/config[3])**2 + (np.cos(config[2])/config[4])**2
    
    A1 = (np.cos(config[7])/config[8])**2 + (np.sin(config[7])/config[9])**2
    B1 = 2 * np.cos(config[7]) * np.sin(config[7]) * ((1/config[8]**2) - (1/config[9]**2))
    C1 = (np.sin(config[7])/config[8])**2 + (np.cos(config[7])/config[9])**2
        
    for i in range(len(train_d_1)):
        x0 = np.real(train_d_1[i]) - config[0]
        y0 = np.imag(train_d_1[i]) - config[1]
        frac_0 = sum(A0*x0**2 + B0*x0*y0 + C0*y0**2 <= 1)
        x1 = np.real(train_d_1[i]) - config[5]
        y1 = np.imag(train_d_1[i]) - config[6]
        frac_1 = sum(A1*x1**2 + B1*x1*y1 + C1*y1**2 <= 1)
        if frac_0 + frac_1 != 0:
            frac_0 /= (frac_0 + frac_1)
            o_pf_up[i] = abs(train_o_p[i]-frac_0)
            o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = None
    perf_dn = None
    if obj == 'median':
        perf_up = np.median(o_pf_up)
        perf_dn = np.median(o_pf_dn)
    elif obj == 'spread':
        perf_up = np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25)
        perf_dn = np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25)
    elif obj == 'mdnspd':
        perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
        perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine(obj, train_d_1, train_o_p):
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config, obj, train_d_1, train_o_p)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config, obj, train_d_1, train_o_p)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list

# for dd in date_dir:
#     train_o_p = np.load(dd + '/train_o_p.npy')
#     train_d_1 = np.load(dd + '/train_d_1.npy')
#     for obj in ['median', 'spread', 'mdnspd']:
#         times = []
#         confs = []
#         perfs = []
#         for k in range(num_runs):
#             start = time.time()
#             config_list, perf_list = simulated_annealing_engine(obj, train_d_1, train_o_p)
#             end = time.time()

#             config = config_list[perf_list.index(min(perf_list))]
#             perf = min(perf_list)

#             print(obj, k, end-start, config, perf)

#             times.append(end-start)
#             confs.append(config)
#             perfs.append(perf)

#         np.save(dd + '/E2D_' + obj + '_runtime.npy', times)
#         np.save(dd + '/E2D_' + obj + '_configs.npy', confs)
#         np.save(dd + '/E2D_' + obj + '_perform.npy', perfs)

## Sensitivity DISQ_E2D data collection

In [5]:
# For sensitivity
date_dir = ['../data3/2021-11-03', '../data3/2021-11-04', '../data3/2021-11-05', '../data3/2021-11-06', '../data3/2021-11-07', '../data3/2021-11-08']
partition = [i/10 for i in range(1,11)]
for dd in date_dir:
    train_o_p = np.load(dd + '/train_o_p.npy')
    train_d_1 = np.load(dd + '/train_d_1.npy')
    for obj in ['median', 'spread', 'mdnspd']:
        out_times = []
        out_confs = []
        out_perfs = []
        
        for part in partition:
            times = []
            confs = []
            perfs = []
            stop = int(part*len(train_o_p)) - 1
            for k in range(num_runs):
                start = time.time()
                config_list, perf_list = simulated_annealing_engine(obj, train_d_1[:stop], train_o_p[:stop])
                end = time.time()

                config = config_list[perf_list.index(min(perf_list))]
                perf = min(perf_list)

                print(obj, k, part, end-start, config, perf)

                times.append(end-start)
                confs.append(config)
                perfs.append(perf)

            # Get the info of the best performing one
            out_confs.append(confs[perfs.index(min(perfs))])
            out_times.append(times[perfs.index(min(perfs))])
            out_perfs.append(min(perfs))
        

        np.save(dd + '/sensitivity_E2D_' + obj + '_runtime.npy', out_times)
        np.save(dd + '/sensitivity_E2D_' + obj + '_configs.npy', out_confs)
        np.save(dd + '/sensitivity_E2D_' + obj + '_perform.npy', out_perfs)

median 0 0.1 1.634009599685669 [-1.6517423538941856, 15.47626028955258, 1.1364636785884203, 5.489391217657438, 4.743960109946991, 2.649811029071868, 11.970287268925835, 0.6961429984290037, 4.362994956166518, 2.3824926003592455] 0.02432300278676406
median 1 0.1 1.6809775829315186 [-4.519253685128305, 14.117781165610168, 1.215979121562211, 4, 5.206725614897149, 5.225503503109016, 11.953804140670533, 5.6046976985514565, 4.441806609587763, 4.462878819976403] 0.010229018741495899
median 2 0.1 1.6159989833831787 [-2.8174699718273155, 15.168933011974921, 5.63416256678499, 6.30309617284262, 5.690534546836666, 4.327698127765786, 17.790954617143353, 5.894643130401067, 2, 4.161177211290583] 0.09240227480620583
median 3 0.1 1.49399995803833 [-4.2221018095881115, 18.35820077269122, 6.103202345889628, 4, 6.249528557532585, 3.405868285501951, 15.518564642090926, 5.949074400254669, 2, 2.513339475801322] 0.01141716457497225
median 4 0.1 1.5149998664855957 [-4.5045915457366785, 16.3398603358633, 4.12754

median 17 0.2 3.5640997886657715 [-4.606870769438373, 13.994330024508765, 4.200761259061503, 8, 5.767294327075478, 3.5100272051443886, 16.00087861590886, 4.464034551096101, 4.281848514147384, 3.275921124903161] 0.03661324371813717
median 18 0.2 3.1621499061584473 [-3.652399092669516, 17.9185110555485, 2.2908372581642604, 6.889291965867658, 6.298769670870717, 3.7045752477680773, 11.702736499113735, 6.108578761679276, 2.982950561998501, 5] 0.026413522412744195
median 19 0.2 3.1873724460601807 [-4.733099066427009, 18.36357759578827, 1.1040670570996296, 7.188181095576534, 4, 6.1506532276140655, 18.336936402990688, 5.573158287401925, 2.2818014346121682, 4.015097923257956] 0.12235470244901375
median 0 0.3 5.15021276473999 [-1.6132117090448665, 17.075429116533392, 0, 5.370069413770244, 4, 4.5963420684803395, 15.836146858312599, 5.670958733651066, 4.342646652602346, 3.6776271717116633] 0.03217511980468272
median 1 0.3 5.1727893352508545 [-0.40623756859144144, 13.567361067318053, 0, 4, 7.332496

median 14 0.4 6.512121677398682 [-3.894400298249953, 18.811993999338455, 1.6994449782993233, 6.044642161931205, 4.670466800125563, 6.680844703582307, 15.926774009428906, 1.8842273885927587, 2, 5] 0.05024863621947845
median 15 0.4 6.563922166824341 [-5, 19.13596209815395, 6.283185307179586, 5.448174795088065, 7.737139962296011, 3.826512667290176, 15.61653504738724, 4.541055115966249, 3.2394572858343142, 2.482011575618064] 0.01561513808217263
median 16 0.4 6.5292627811431885 [-5, 15.213815801980314, 5.3957959763691505, 6.771505774851792, 4, 4.928195136471654, 17.036645077865785, 1.7779033438419192, 4.939015275998838, 5] 0.020771668708464176
median 17 0.4 6.550529718399048 [-4.687683234459084, 13.407202655768515, 5.988381586826458, 5.78258492013112, 4.910819287499185, 6.334712795109048, 15.036580587029633, 3.0910323233647423, 4.429296042077362, 4.515968199403142] 0.05119038992004438
median 18 0.4 6.606816291809082 [-4.714950139142177, 17.211661063497605, 3.17596365865453, 6.21751216758493

median 11 0.6 9.935180902481079 [-3.563246823713939, 12.424571671842282, 4.727629607681527, 7.077598557780569, 4, 1.6567672976213241, 16.267733203793817, 3.0946914369313037, 3.0411309304128933, 4.059325504179864] 0.04473658710240869
median 12 0.6 10.039540529251099 [-0.2748708489528455, 10.397460797475066, 0, 4, 5.71613276891082, 3.7749710334843303, 15.370915149003176, 0.18712419825844573, 2.3382004023166774, 3.467294072229687] 0.03911008528990351
median 13 0.6 10.3052237033844 [-2.8969288513547142, 19.02565495689634, 3.4177865335457165, 7.766139038289941, 5.134939846367519, 5.78775116028314, 14.58015591053032, 0.5969647303165716, 3.154538971820113, 4.082619347978476] 0.027288317665323525
median 14 0.6 10.052847623825073 [-5, 10.316966786077236, 4.352319506648742, 7.453428136053683, 4.598789769676026, 1.17051345456323, 11.27540279364364, 1.7837703583311464, 3.3751708326976217, 4.3752853138845555] 0.05810107456102831
median 15 0.6 9.889188051223755 [0, 18.891779633929147, 3.083813899968

median 8 0.8 13.212481260299683 [-0.5434183559159618, 18.45747613709879, 4.857315607889547, 4, 5.6512492438749256, 8.479325237939117, 12.746532193555284, 1.3414214616058968, 4.906857326989802, 5] 0.1547592221811988
median 9 0.8 13.249313592910767 [-2.1072182339626084, 13.425384162979755, 1.001250731630548, 4, 7.4513614056750805, 2.5317632861729145, 14.535867090069546, 1.135290637612613, 2.8495343470417014, 3.161903132328518] 0.10460049643319147
median 10 0.8 13.290724515914917 [-4.6589274172099975, 20, 3.227423431533704, 4.64948012382853, 8, 3.626038740190647, 16.751740315698864, 3.585274312983801, 3.2020133664507995, 4.9018971475288] 0.009623069312939658
median 11 0.8 13.43596339225769 [-2.7936322962907068, 12.136295726647733, 0.9359726723071218, 6.937704741994501, 6.5326175642282465, 4.535602190748872, 14.557555934348002, 2.443864008832728, 4.012137488365382, 4.671963771918259] 0.12327176858660599
median 12 0.8 13.648385047912598 [-1.5704245014159837, 20, 5.916432654659321, 4.5256356

...

mdnspd 18 0.2 5.954137802124023 [-1.683508517365768, 17.618336665280367, 5.016133598478464, 4.8636327293236015, 8, 7.658339788218574, 14.767173502398716, 1.9544424164988299, 5, 3.1550672788021537] 1.2150417705882273
mdnspd 19 0.2 5.945783853530884 [-3.5450595936952016, 10, 0.6530642097418586, 4.382839820253324, 7.090495843452958, 1.2845902454079408, 12.36552896548701, 1.6157992252768543, 2.595376539800651, 2] 0.07458236218689557
mdnspd 0 0.3 9.24965214729309 [-0.34193178334516383, 16.27309891980728, 0.9628953598072107, 4.801831420703119, 7.723848175567536, 3.049224594860407, 10.779738402982732, 0.6104271153146379, 4.013574053526691, 4.686476431585687] 0.5420642950333182
mdnspd 1 0.3 9.213721513748169 [-2.3641887451715364, 19.25648242694129, 0, 6.796311651815092, 6.0309425849646505, 2.8290301523414896, 16.258650848567733, 1.7836393621862474, 4.362175800455192, 4.104662046560774] 0.16059367220858256
mdnspd 2 0.3 9.112947225570679 [-3.2835638825644153, 19.629108782552937, 5.93161670006452

mdnspd 14 0.4 11.423343658447266 [-0.62006470696396, 19.5322590081117, 6.283185307179586, 4.23934857412123, 4, 10, 14.470604868262338, 1.7024581096798175, 2.6998393206666966, 5] 0.9710403604325769
mdnspd 15 0.4 11.027166366577148 [-3.555502427174228, 20, 6.263811947789188, 5.206543585893287, 6.989717608947735, 4.48056192803778, 13.450094530841167, 0.9086554623497725, 2.4244922248208596, 4.659461755401763] 0.04699680266690304
mdnspd 16 0.4 9.38125491142273 [-3.7304372977845786, 15.197265268269975, 6.175591123024757, 4, 7.601378897057418, 0.6708830064495652, 10.497776897571226, 6.283185307179586, 5, 4.001177884734081] 0.16011766458282642
mdnspd 17 0.4 11.200145721435547 [-1.4250300166204066, 15.083842235954389, 2.6151468487224174, 4.528009330209763, 7.193773105126038, 2.5192592089435966, 13.081820973358457, 4.762087370557826, 3.5237175059330346, 5] 0.34901272388019444
mdnspd 18 0.4 11.495011806488037 [-4.504921884558133, 18.36792143907468, 5.178522699746027, 4, 8, 3.5943586073904408, 17.

mdnspd 11 0.6 14.447418928146362 [-3.2729979646009006, 19.463897427897695, 4.378479022617867, 7.889755054711549, 4.599318409952215, 3.0863179824356637, 10.724441225570956, 4.160255090064332, 4.294687807026066, 4.473337145866678] 0.04050634535060807
mdnspd 12 0.6 12.507020235061646 [-5, 10.411956679170402, 4.030439170909241, 4, 7.550022577528637, 2.3472508499321796, 16.879543613311974, 1.6813278328930423, 2.7118107453524143, 3.8021530350666746] 0.07338932723259937
mdnspd 13 0.6 12.224020004272461 [-0.5614665238889842, 19.769800712113828, 4.888840764361307, 4.236312539201344, 6.086194641215736, 3.705508907855789, 16.21951228900442, 1.8976958780154443, 2.699262347658587, 2] 0.23324290263584013
mdnspd 14 0.6 11.995658874511719 [-4.913849838119135, 19.141065477312235, 3.5416829796206386, 7.405034309868755, 5.887913849212767, 6.801090004434084, 13.653394185789075, 3.5690715828933115, 4.807205911893489, 5] 0.04603394583014088
mdnspd 15 0.6 14.644487142562866 [-5, 15.65604051301979, 3.79817570

mdnspd 8 0.8 24.068328380584717 [-0.6695403959907382, 16.020814111171802, 1.0000622087592677, 6.2175935612967566, 4, 3.093863372420455, 13.359169072091182, 1.4285857860748192, 2.8428338639714585, 2.966311812937761] 0.3300678370734747
mdnspd 9 0.8 24.52634906768799 [-4.6223318334834556, 10, 1.8014892175843065, 7.332833782094048, 5.983986404561183, 0.7327540371635552, 10.165282348963844, 4.182558177054579, 5, 4.260498428710081] 0.11143466287615195
mdnspd 10 0.8 25.26016330718994 [-1.9331979018387715, 12.211399189394793, 0, 8, 5.950962840364541, 5.778107781220617, 17.04337574488622, 5.35316583391001, 2.6911413832092324, 4.225474384902421] 0.7718694116887455
mdnspd 11 0.8 25.41193175315857 [-5, 19.272309863084196, 1.0489285157410624, 7.603812920341942, 4, 5.332164615015171, 16.85881655390329, 5.02484938792799, 2.417048052377987, 4.282655824429388] 0.03398948189249719
mdnspd 12 0.8 25.306322813034058 [-1.5059061680021375, 20, 6.283185307179586, 7.489917194045659, 4.7837373072841505, 7.72852

mdnspd 5 1.0 28.00708031654358 [-4.341220832204238, 18.421161575173794, 0, 7.669101373204903, 4.1169316311414805, 3.893118106296847, 15.988824482586082, 6.283185307179586, 2, 2.9598362181914597] 0.11199045926978655
mdnspd 6 1.0 25.810391902923584 [-4.096698896939696, 15.067230705885374, 2.2920354625014685, 8, 7.474089914919365, 7.239291029154254, 12.165494556895977, 2.945300393877256, 4.750890910352636, 2] 0.7912564364397647
mdnspd 7 1.0 26.90312886238098 [-2.658157292895702, 10.916077935001061, 0.41502080795822627, 5.964056615834713, 5.384159086317069, 3.223205865928956, 16.76699616118353, 3.210866924080877, 3.526555605059335, 3.7490439063478576] 0.3654856450850227
mdnspd 8 1.0 27.039302110671997 [-2.7976108469860526, 14.815523962325756, 2.1598909195221845, 4, 7.494947042625131, 1.1426471286902593, 12.851717630903961, 1.8409906772753422, 5, 3.029129326636248] 0.2588555432471755
mdnspd 9 1.0 26.75611138343811 [-3.941703982629831, 10, 6.051990211552785, 6.123922445282925, 4, 3.129062569

In [11]:
# Discriminant number of parameters and bounds
# For Cir2D: origin-x, origin-y, radius X 2
num_params = 6

# The first 3 elements are for state 0 -> origin-x, origin-y, radius x 2
# The last 3 elements are for state 1 -> origin-x, origin-y, radius x 2
bounds = [[-5, 0], [10, 20], [4, 8], [0, 10], [10, 20], [4, 8]]
# The bounds are based on observation mainly


# Sample the performance of a configuration
def sample_perf(config, obj):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        
        # frac_0 is the number of points that can be classified as "0" as it is inside the circle with origin_x config[0]
        # origin_y config[1] and radius config[2]
        frac_0 = sum(np.absolute(train_d_1[i] - complex(config[0], config[1])) <= config[2])
#         print(frac_0)
        
        # This is raw totals, as in it sums the points that can be classified as "1" as it is inside the circle with origin_x
        # config [3], origin_y config[4] and radius config[5]. The "totals" here is the total number of classifiable points
        totals = frac_0 + sum(np.absolute(train_d_1[i] - complex(config[3], config[4])) <= config[5])
#         print(totals)
        
        if totals != 0:
            frac_0 /= totals
            o_pf_up[i] = abs(train_o_p[i]-frac_0)
            o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = None
    perf_dn = None
#     print("O_pf_UP")
#     print(o_pf_up)
#     print("O_pf_DN")
#     print(o_pf_dn)
    if obj == 'median':
        perf_up = np.median(o_pf_up)
        perf_dn = np.median(o_pf_dn)
    elif obj == 'spread':
        perf_up = np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25)
        perf_dn = np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25)
    elif obj == 'mdnspd':
        perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
        perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine(obj):
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config, obj)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config, obj)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list

for obj in ['median', 'spread', 'mdnspd']:
    times = []
    confs = []
    perfs = []
    for k in range(num_runs):
        start = time.time()
        config_list, perf_list = simulated_annealing_engine(obj)
        end = time.time()

        config = config_list[perf_list.index(min(perf_list))]
        perf = min(perf_list)

        print(obj, k, end-start, config, perf)

        times.append(end-start)
        confs.append(config)
        perfs.append(perf)

    np.save(date_dir + '/C2D_' + obj + '_runtime.npy', times)
    np.save(date_dir + '/C2D_' + obj + '_configs.npy', confs)
    np.save(date_dir + '/C2D_' + obj + '_perform.npy', perfs)

median 0 18.558857917785645 [-5, 16.96989387044581, 5.801931675100565, 5.325992888601659, 15.81481405179559, 4.997533453363176] 0.015948660592974978
median 1 21.1056809425354 [-3.784870534355101, 13.145745296304248, 4, 8.140161235385147, 14.020852207598331, 5.810514453500044] 0.06266476459529186
median 2 17.377114295959473 [-2.5304054201059722, 19.193808734474885, 6.020433820306343, 4.9980500295142125, 11.623169674337277, 4] 0.0358347516557488
median 3 18.429208517074585 [-2.138583173482926, 10, 5.704705493416352, 2.175053017840867, 13.722894323329578, 4.54231467741521] 0.13799112115077125
median 4 16.784940004348755 [-4.7481862028181325, 14.767834808142076, 7.1495764608044405, 8.105819493727363, 16.768423120348427, 8] 0.0771590743204637
median 5 17.032823085784912 [-1.6179391593335475, 10, 4.042360355572509, 9.473511537486862, 10.960571096792961, 6.976655990090777] 0.03291401942566191
median 6 18.639418125152588 [-1.9091233507386685, 11.76185679477757, 4.169777876433542, 4.83505361219

mdnspd 13 16.66942286491394 [-4.747631033731878, 17.61337626206469, 4.489866857102403, 4.455184894569588, 18.499068537532313, 5.081884097480143] 0.04065059292635896
mdnspd 14 16.749555587768555 [-2.531576767863962, 10, 5.617367770163908, 3.250272612803054, 17.887681102460068, 5.944965771827442] 0.20561821791972773
mdnspd 15 16.660144567489624 [-3.8323478328880602, 14.661801462380161, 4, 4.30769514754613, 17.015898662665425, 4.772045368633988] 0.051818190416449875
mdnspd 16 16.92370629310608 [-2.325640500383911, 17.77189981897206, 4.64872561517306, 6.732105250459127, 13.452197913028668, 4.388254026073155] 0.1026453064262504
mdnspd 17 16.74739122390747 [-2.5110620138736994, 11.513979252565218, 5.022403370859965, 10, 15.15857781146698, 7.589202851584888] 0.30554708162792876
mdnspd 18 16.681697845458984 [-2.9942712462163534, 18.630543373359497, 4.874501480064877, 7.984640710017384, 15.378137088255622, 5.7306964606058814] 0.0433199946458239
mdnspd 19 16.72501802444458 [-5, 17.43002102511158

In [ ]:
'''
# Discriminant number of parameters and bounds
# For LinD: origin-x, origin-y, radius
num_params = 2
bounds = [[0, 5], [-25, 0]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = 0
        for s in train_d_1[i]:
            y = config[0]*np.real(s) + config[1]
            if np.imag(s) >= y:
                frac_0 += 1
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/LinD_runtimes.npy', times)
np.save(date_dir + '/LinD_configurations.npy', confs)
np.save(date_dir + '/LinD_performances.npy', perfs)
'''

In [ ]:
'''
# Discriminant number of parameters and bounds
# For QuaD: origin-x, origin-y, radius
num_params = 4
bounds = [[0, 20], [-10, 10], [0, 2*np.pi], [0.1, 5]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = 0
        for s in train_d_1[i]:
            x = np.real(s) - config[0]
            y = np.imag(s) - config[1]
            r = np.sqrt(x**2 + y**2)
            t = np.arctan2(y, x) - config[2]
            x = (r * np.cos(t)) + config[0]
            y = (r * np.sin(t)) + config[1]
            if y >= (config[3]*((x-config[0])**2)+config[1]):
                frac_0 += 1
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/QuaD_runtimes.npy', times)
np.save(date_dir + '/QuaD_configurations.npy', confs)
np.save(date_dir + '/QuaD_performances.npy', perfs)
'''

In [ ]:
'''
# Discriminant number of parameters and bounds
# For CirD: origin-x, origin-y, radius
num_params = 3
bounds = [[-8, 10], [-10, 10], [5, 10]]

# Sample the performance of a configuration
def sample_perf(config):

    o_pf_up = [1]*len(train_d_1)
    o_pf_dn = [1]*len(train_d_1)
    for i in range(len(train_d_1)):
        frac_0 = sum(np.absolute(train_d_1[i] - complex(config[0], config[1])) <= config[2])
        frac_0 /= num_shots
        o_pf_up[i] = abs(train_o_p[i]-frac_0)
        o_pf_dn[i] = abs(train_o_p[i]-(1-frac_0))

    perf_up = np.median(o_pf_up) + (np.percentile(o_pf_up, 75) - np.percentile(o_pf_up, 25))
    perf_dn = np.median(o_pf_dn) + (np.percentile(o_pf_dn, 75) - np.percentile(o_pf_dn, 25))
        
    return min(perf_up, perf_dn)

# Declare the simulated annealing engine
def simulated_annealing_engine():
    
    cur_config = [0]*num_params
    for i in range(num_params):
        cur_config[i] = np.random.uniform(bounds[i][0], bounds[i][1])
    min_config = [c for c in cur_config]
    min_perf = sample_perf(min_config)
    cur_temp = ini_temp
    
    config_list = [[0]*num_params]*num_sa_iters
    perf_list = [0]*num_sa_iters

    accept = False
    for i in range(num_sa_iters):
        for j in range(num_params):
            cur_config[j] = min_config[j] + np.random.uniform(-1, 1)
            cur_config[j] = max(min(cur_config[j], bounds[j][1]), bounds[j][0])

        cur_perf = sample_perf(cur_config)
        E = min_perf - cur_perf

        if E < 0:
            p = np.exp(E / cur_temp)
            if np.random.random() < p:
                accept = True
            else:
                accept = False
        else:
            accept = True

        if accept == True:
            min_config = [c for c in cur_config]
            min_perf = cur_perf
        
        config_list[i] = [c for c in min_config]
        perf_list[i] = min_perf
        
        # Cooling the temperature
        cur_temp *= cool_coef
        
    return config_list, perf_list
            
times = []
confs = []
perfs = []
for k in range(num_runs):
    start = time.time()
    config_list, perf_list = simulated_annealing_engine()
    end = time.time()

    config = config_list[perf_list.index(min(perf_list))]
    perf = min(perf_list)
    
    print(k, end-start, config, perf)
    
    times.append(end-start)
    confs.append(config)
    perfs.append(perf)

np.save(date_dir + '/CirD_runtimes.npy', times)
np.save(date_dir + '/CirD_configurations.npy', confs)
np.save(date_dir + '/CirD_performances.npy', perfs)
'''